# Set up CUDA 

- How To Run CUDA C or C++ on Google Colab or Azure Notebook [medium](https://harshityadav95.medium.com/how-to-run-cuda-c-or-c-on-google-colab-or-azure-notebook-ea75a23a5962)

Check the procesor infomation

In [1]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

Script to setup the CUDA environment

In [2]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

Check the Version of CUDA

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


Query the GPU device state

In [4]:
!nvidia-smi

Fri Mar  5 11:29:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

printblockid.cu

In [10]:
%%cu
#include <stdio.h>

#define NUM_BLOCKS 16
#define BLOCK_WIDTH 1

__global__ void hello()
{
    printf("Hello world! I'm a thread in block %d\n", blockIdx.x);
}


int main(int argc,char **argv)
{
    // launch the kernel
    hello<<<NUM_BLOCKS, BLOCK_WIDTH>>>();

    // force the printf()s to flush
    cudaDeviceSynchronize();

    printf("That's all!\n");
	system("pause");
    return 0;
}

Hello world! I'm a thread in block 2
Hello world! I'm a thread in block 12
Hello world! I'm a thread in block 7
Hello world! I'm a thread in block 4
Hello world! I'm a thread in block 14
Hello world! I'm a thread in block 9
Hello world! I'm a thread in block 0
Hello world! I'm a thread in block 10
Hello world! I'm a thread in block 3
Hello world! I'm a thread in block 13
Hello world! I'm a thread in block 5
Hello world! I'm a thread in block 1
Hello world! I'm a thread in block 8
Hello world! I'm a thread in block 15
Hello world! I'm a thread in block 11
Hello world! I'm a thread in block 6
sh: 1: pause: not found
That's all!



MatrixMul1Block.cu

In [5]:
%%cu
#include <cuda.h>
#include <iostream>
#include <time.h>       //clock()
using namespace std;

__global__ void MatrixMulKernel( int m, int n, int k, float* d_A, float* d_B, float* d_C) { 
	
	int row = threadIdx.x;
    int col = threadIdx.y;
	float sum,a,b; 
	
	sum = 0;
    for (int i = 0; i < n; ++i) {
        a = d_A[row * n + k];
        b = d_B[k * i + col];
	    sum += a * b;
    }
    d_C[row * n + col] = sum;
     
}

void InitMatrix(float * M,  int numMRows, int numMColumns){
	int i, j;
	for(i = 0; i < numMRows; i ++)
		for(j = 0; j < numMColumns; j ++)
			M[i* numMColumns+j] = 1;
}
void PrintMatrix(float * M,  int numMRows, int numMColumns){
	int i, j;
	for(i = numMRows-10; i < numMRows; i ++) {
		cout << endl;
		for(j = numMColumns-10; j < numMColumns; j ++)
			cout << M[i* numMColumns+j] << " ";
	}
}

int main(int argc, char ** argv) {
	
	clock_t Tclock_start, Tclock_end, clock_start,clock_end;
	Tclock_start = clock();
	
    float * h_A; // The A matrix
    float * h_B; // The B matrix
    float * h_C; // The output C matrix
    float * d_A;
    float * d_B;
    float * d_C;
    int numARows; // number of rows in the matrix A
    int numAColumns; // number of columns in the matrix A
    int numBRows; // number of rows in the matrix B
    int numBColumns; // number of columns in the matrix B
    int numCRows; // number of rows in the matrix C (you have to set this)
    int numCColumns; // number of columns in the matrix C (you have to set this)

	cout << "Importing data and creating memory on host. " ;
	clock_start = clock();
	
	int nn =32;
	numARows = nn; // number of rows in the matrix A
    numAColumns = nn; // number of columns in the matrix A
    numBRows = nn; // number of rows in the matrix B
	numBColumns = nn; // number of columns in the matrix B
	
	int m = numARows;
	int n = numAColumns;
	int k = numBColumns;

	//@@ Allocate and initialize hostA and hostB 
	int sizeA = numARows * numAColumns * sizeof(float);
	h_A = (float *) malloc(sizeA);
	int sizeB = numBRows * numBColumns * sizeof(float);
    h_B = (float *) malloc(sizeB);
    
	InitMatrix(h_A,numARows,numAColumns);
	InitMatrix(h_B,numBRows,numBColumns);
	
	
	//@@ Set numCRows and numCColumns
    numCRows = numARows;
    numCColumns = numBColumns;

    //@@ Allocate the hostC matrix
	int sizeC = numCRows * numCColumns * sizeof(float);
    h_C = (float *) malloc(sizeC);
    
	clock_end = clock();
	cout << clock_end-clock_start << " miliseconds"<< endl; 
    
	cout << "Allocating GPU memory. " ;
	clock_start = clock();
	//@@ Allocate GPU memory here
    cudaMalloc((void **) &d_A, sizeA);
	cudaMalloc((void **) &d_B, sizeB);
	cudaMalloc((void **) &d_C, sizeC);
	clock_end = clock();
	cout << clock_end-clock_start << " miliseconds"<< endl; 
	
	cout << "Copying input memory to the GPU. " ;
	clock_start = clock();
    //@@ Copy memory to the GPU here
    cudaMemcpy(d_A, h_A, sizeA, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, sizeB, cudaMemcpyHostToDevice);
	clock_end = clock();
	cout << clock_end-clock_start << " miliseconds"<< endl; 
	
	//@@ Initialize the grid and block dimensions here
    dim3 dimGrid(1, 1, 1);
	dim3 dimBlock(nn, nn, 1);
	
    
	cout << "Performing CUDA computation. " ;
	clock_start = clock();
    //@@ Launch the GPU Kernel here
    MatrixMulKernel<<<dimGrid, dimBlock>>> (m, n, k, d_A, d_B, d_C);  
    cudaThreadSynchronize();
 	clock_end = clock();
	cout << clock_end-clock_start << " miliseconds"<< endl; 
	
	cout << "Copying output memory to the CPU. " ;
	clock_start = clock();
    //@@ Copy the GPU memory back to the CPU here
    cudaMemcpy(h_C, d_C, sizeC, cudaMemcpyDeviceToHost);
 	clock_end = clock();
	cout << clock_end-clock_start << " miliseconds"<< endl; 
	
    cout << "Freeing GPU Memory. " ;
	clock_start = clock();
    //@@ Free the GPU memory here
    cudaFree(d_A); 
    cudaFree(d_B); 
    cudaFree(d_C);
  	clock_end = clock();
	cout << clock_end-clock_start << " miliseconds"<< endl; 

	cout << endl <<  "The dimensions of A are " << numARows << " x " << numAColumns ;
	PrintMatrix(h_A,numARows,numAColumns);
	
	cout << endl<<  "The dimensions of B are " << numBRows << " x " << numBColumns;
	PrintMatrix(h_B,numBRows,numBColumns);
	
	cout << endl<<  "The dimensions of C are " << numCRows << " x " << numCColumns;
	PrintMatrix(h_C,numCRows,numCColumns);
    
    free(h_A);
    free(h_B);
    free(h_C);

	Tclock_end = clock();
    cout << endl << "---------------------------------------------------" << endl;    
    cout << endl << "Time elapsed  is " << Tclock_end-Tclock_start << "  miliseconds"<< endl; 
    cout << endl << "---------------------------------------------------" << endl;    

	system("pause");
    return 0;
}




Importing data and creating memory on host. 5 miliseconds
Allocating GPU memory. 186007 miliseconds
Copying input memory to the GPU. 35 miliseconds
Performing CUDA computation. 44 miliseconds
Copying output memory to the CPU. 23 miliseconds
Freeing GPU Memory. 108 miliseconds

The dimensions of A are 32 x 32
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
The dimensions of B are 32 x 32
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
The dimensions of C are 32 x 32
32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32 32 32 32 
32 32 32 32 32 32 32

MatrixMulGrid.cu

In [6]:
%%cu
#include <cuda.h>
#include <iostream>
#include <time.h>       //clock()
using namespace std;

__global__ void MatrixMulKernel( int m, int n, int k, float* d_A, float* d_B, float* d_C) { 
	
	int row = blockIdx.x;
    int col = blockIdx.y;
	float sum,a,b; 
	
	sum = 0;
    for (int i = 0; i < n; ++i) {
        a = d_A[row * n + k];
        b = d_B[k * i + col];
	    sum += a * b;
    }
    d_C[row * n + col] = sum;
     
}

void InitMatrix(float * M,  int numMRows, int numMColumns){
	int i, j;
	for(i = 0; i < numMRows; i ++)
		for(j = 0; j < numMColumns; j ++)
			M[i* numMColumns+j] = 1;
}
void PrintMatrix(float * M,  int numMRows, int numMColumns){
	int i, j;
	for(i = numMRows-10; i < numMRows; i ++) {
		cout << endl;
		for(j = numMColumns-10; j < numMColumns; j ++)
			cout << M[i* numMColumns+j] << " ";
	}
}

int main(int argc, char ** argv) {
	
	clock_t Tclock_start, Tclock_end, clock_start,clock_end;
	Tclock_start = clock();
	
    float * h_A; // The A matrix
    float * h_B; // The B matrix
    float * h_C; // The output C matrix
    float * d_A;
    float * d_B;
    float * d_C;
    int numARows; // number of rows in the matrix A
    int numAColumns; // number of columns in the matrix A
    int numBRows; // number of rows in the matrix B
    int numBColumns; // number of columns in the matrix B
    int numCRows; // number of rows in the matrix C (you have to set this)
    int numCColumns; // number of columns in the matrix C (you have to set this)

	cout << "Importing data and creating memory on host. " ;
	clock_start = clock();
	
	int nn = 435;
	numARows = nn; // number of rows in the matrix A
    numAColumns = nn; // number of columns in the matrix A
    numBRows = nn; // number of rows in the matrix B
	numBColumns = nn; // number of columns in the matrix B
	
	int m = numARows;
	int n = numAColumns;
	int k = numBColumns;

	//@@ Allocate and initialize hostA and hostB 
	int sizeA = numARows * numAColumns * sizeof(float);
	h_A = (float *) malloc(sizeA);
	int sizeB = numBRows * numBColumns * sizeof(float);
    h_B = (float *) malloc(sizeB);
    
	InitMatrix(h_A,numARows,numAColumns);
	InitMatrix(h_B,numBRows,numBColumns);
	
	
	//@@ Set numCRows and numCColumns
    numCRows = numARows;
    numCColumns = numBColumns;

    //@@ Allocate the hostC matrix
	int sizeC = numCRows * numCColumns * sizeof(float);
    h_C = (float *) malloc(sizeC);
    
	clock_end = clock();
	cout << clock_end-clock_start << " miliseconds"<< endl; 
    
	cout << "Allocating GPU memory. " ;
	clock_start = clock();
	//@@ Allocate GPU memory here
    cudaMalloc((void **) &d_A, sizeA);
	cudaMalloc((void **) &d_B, sizeB);
	cudaMalloc((void **) &d_C, sizeC);
	clock_end = clock();
	cout << clock_end-clock_start << " miliseconds"<< endl; 
	
	cout << "Copying input memory to the GPU. " ;
	clock_start = clock();
    //@@ Copy memory to the GPU here
    cudaMemcpy(d_A, h_A, sizeA, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, sizeB, cudaMemcpyHostToDevice);
	clock_end = clock();
	cout << clock_end-clock_start << " miliseconds"<< endl; 
	
	//@@ Initialize the grid and block dimensions here
    dim3 dimGrid(nn,nn, 1);
	dim3 dimBlock(1, 1, 1);
	
    
	cout << "Performing CUDA computation. " ;
	clock_start = clock();
    //@@ Launch the GPU Kernel here
    MatrixMulKernel<<<dimGrid, dimBlock>>> (m, n, k, d_A, d_B, d_C);  
    cudaThreadSynchronize();
 	clock_end = clock();
	cout << clock_end-clock_start << " miliseconds"<< endl; 
	
	cout << "Copying output memory to the CPU. " ;
	clock_start = clock();
    //@@ Copy the GPU memory back to the CPU here
    cudaMemcpy(h_C, d_C, sizeC, cudaMemcpyDeviceToHost);
 	clock_end = clock();
	cout << clock_end-clock_start << " miliseconds"<< endl; 
	
    cout << "Freeing GPU Memory. " ;
	clock_start = clock();
    //@@ Free the GPU memory here
    cudaFree(d_A); 
    cudaFree(d_B); 
    cudaFree(d_C);
  	clock_end = clock();
	cout << clock_end-clock_start << " miliseconds"<< endl; 

	cout << endl <<  "The dimensions of A are " << numARows << " x " << numAColumns ;
	PrintMatrix(h_A,numARows,numAColumns);
	
	cout << endl<<  "The dimensions of B are " << numBRows << " x " << numBColumns;
	PrintMatrix(h_B,numBRows,numBColumns);
	
	cout << endl<<  "The dimensions of C are " << numCRows << " x " << numCColumns;
	PrintMatrix(h_C,numCRows,numCColumns);
    
    free(h_A);
    free(h_B);
    free(h_C);

	Tclock_end = clock();
    cout << endl << "---------------------------------------------------" << endl;    
    cout << endl << "Time elapsed  is " << Tclock_end-Tclock_start << "  miliseconds"<< endl; 
    cout << endl << "---------------------------------------------------" << endl;    

	system("pause");
    return 0;
}




Importing data and creating memory on host. 1324 miliseconds
Allocating GPU memory. 177353 miliseconds
Copying input memory to the GPU. 453 miliseconds
Performing CUDA computation. 7853 miliseconds
Copying output memory to the CPU. 573 miliseconds
Freeing GPU Memory. 244 miliseconds

The dimensions of A are 435 x 435
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
The dimensions of B are 435 x 435
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
The dimensions of C are 435 x 435
435 435 435 435 435 435 435 435 435 435 
435 435 435 435 435 435 435 435 435 435 
435 435 435 435 435 435 435 435 435 435 
435 435 435 435 435 435 435 435 435 435 
435 435 435 435 435 435 435 4

MultiBlockMultiThreadMatrixMultiplication.cu

In [7]:
%%cu
#define TILE_WIDTH 32 
#define GPU_MAT_SIZE 2000
#include <cuda.h>
#include <iostream>
#include <time.h>       //clock()
using namespace std;

__global__ void MatrixMulKernel( float* d_A, float* d_B, float* d_C, int m, int n, int k) { 
	
	int Row = blockIdx.y*blockDim.y + threadIdx.y;
    int Col = blockIdx.x*blockDim.x + threadIdx.x;
	
    if ((Row < m) && (Col < k)) {
       float Cvalue = 0.0;
       for (int i = 0; i < n; ++i)
            Cvalue += d_A[Row*n+i] * d_B[Col+i*k];
       d_C[Row*k+Col] = Cvalue;
     }
	     
}


void PrintMatrix(float * M,  int numMRows, int numMColumns){
	int i, j;
	for(i = numMRows-10; i < numMRows; i ++) {
		cout << endl;
		for(j = numMColumns-10; j < numMColumns; j ++)
			cout << M[i* numMColumns+j] << " ";
	}
}

void constantInit(float *data, int size, float val)
{
    for (int i = 0; i < size; ++i)
    {
        data[i] = val;
    }
}

void subMat(float *dest, float *data,int n, int m,  int i, int j, int s)
{
    for (int k = 0; k < s; ++k)
    {
			for (int l = 0; l < s; ++l)
			{
				if ( ((k+i*s) < n)  & ((l+j*s) < m) ) { 
					dest[k*s + l] = data[((k+i*s)*m)+(l+j*s)];
				}
				else 
				{
					dest[k*s + l] = 0.0; 
				}
			}
    }
}

void putMat(float *dest, int n, int m, float *data, int i, int j, int s)
{
    for (int k = 0; k < s; ++k)
    {
			for (int l = 0; l < s; ++l)
			{
				if ( ((k+i*s) < n)  & ((l+j*s) < m) ) dest[((k+i*s)*m)+(l+j*s)] = data[k*s + l];
			}
    }
}

void addMem(float *dest, float *data1, float *data2, int size)
{
    for (int i = 0; i < size; ++i)
    {
        dest[i] = data1[i] + data2[i];
    }
}

void GPUsimpleMult(float* h_A, float* h_B, float* h_C, int m, int n, int k){
	
	float * d_A;
    float * d_B;
    float * d_C;

	int size_d_A = m * n * sizeof(float);
	int size_d_B = m * k * sizeof(float);
	int size_d_C = m * k * sizeof(float);

	
	//@@ Allocate GPU memory here
    cudaMalloc((void **) &d_A, size_d_A);
	cudaMalloc((void **) &d_B, size_d_B);
	cudaMalloc((void **) &d_C, size_d_C);
	
    //@@ Copy memory to the GPU here
    cudaMemcpy(d_A, h_A, size_d_A, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size_d_B, cudaMemcpyHostToDevice);
	
	//@@ Initialize the grid and block dimensions here
    dim3 dimGrid((m-1)/TILE_WIDTH+1, (k-1)/TILE_WIDTH+1, 1);
    dim3 dimBlock(TILE_WIDTH, TILE_WIDTH, 1);
	
    //@@ Launch the GPU Kernel here
    MatrixMulKernel<<<dimGrid, dimBlock>>>(d_A,d_B,d_C,m,n,k);  
    cudaThreadSynchronize();
 	
	
	
    //@@ Copy the GPU memory back to the CPU here
    cudaMemcpy(h_C, d_C, size_d_C, cudaMemcpyDeviceToHost);
	
    //@@ Free the GPU memory here
    cudaFree(d_A); 
    cudaFree(d_B); 
    cudaFree(d_C);
  	
}


void GPUroundsMult(float* h_A, float* h_B, float* h_C, int m, int n, int k){
	
	float * d_A;
    float * d_B;
    float * d_C;

	float * h_A_aux; 
	float * h_B_aux; 
	float * h_C_aux0; 
	float * h_C_aux1; 

	int size_d = GPU_MAT_SIZE * GPU_MAT_SIZE * sizeof(float);
	
	//@@ Allocate GPU memory here
	h_A_aux = (float *) malloc( size_d);
	h_B_aux = (float *) malloc( size_d);
	h_C_aux0 = (float *) malloc( size_d);
	h_C_aux1 = (float *) malloc( size_d);
    
	cudaMalloc((void **) &d_A, size_d);
	cudaMalloc((void **) &d_B, size_d);
	cudaMalloc((void **) &d_C, size_d);
	
	for (int i = 0; i < (m-1)/GPU_MAT_SIZE+1; ++i){ 
		
		for (int j = 0; j < (k-1)/GPU_MAT_SIZE+1; ++j){
			
			constantInit(h_C_aux0, GPU_MAT_SIZE * GPU_MAT_SIZE, 0.0);

			for (int l = 0; l < (n-1)/GPU_MAT_SIZE+1; ++l){

					subMat(h_A_aux, h_A,n,m, i, l, GPU_MAT_SIZE);
					subMat(h_B_aux, h_B,m,k, l, j, GPU_MAT_SIZE);
					GPUsimpleMult(h_A_aux, h_B_aux, h_C_aux1, GPU_MAT_SIZE,GPU_MAT_SIZE,GPU_MAT_SIZE);
					addMem(h_C_aux0,h_C_aux0,h_C_aux1,GPU_MAT_SIZE * GPU_MAT_SIZE);
			}

			putMat(h_C, m, k, h_C_aux0, i, j,GPU_MAT_SIZE );
		}       
	}

	//@@ Free the aux CPU and GPU memory here
    cudaFree(d_A); 
    cudaFree(d_B); 
    cudaFree(d_C);
  	free(h_A_aux);
    free(h_B_aux);
    free(h_C_aux0);
	free(h_C_aux1);

}
int main(int argc, char ** argv) {
	clock_t clock_start,clock_end;
    clock_start = clock();

	float * h_A; // The A matrix
    float * h_B; // The B matrix
    float * h_C; // The output C matrix
    
    int rows_h_A; // number of rows in the matrix A
    int columns_h_A; // number of columns in the matrix A
    int rows_h_B; // number of rows in the matrix B
    int columns_h_B; // number of columns in the matrix B
    int rows_h_C; // number of rows in the matrix C (you have to set this)
    int columns_h_C; // number of columns in the matrix C (you have to set this)
	
	int n = 10000;
	rows_h_A = n; // number of rows in the matrix A
    columns_h_A = n; // number of columns in the matrix A
    rows_h_B = n; // number of rows in the matrix B
	columns_h_B = n; // number of columns in the matrix B
	
     cout << endl << "Allocate, initialize host and device memory " ;
	//@@ Allocate host memory
	int size_h_A = rows_h_A * columns_h_A * sizeof(float);
	h_A = (float *) malloc(size_h_A);
	int size_h_B = rows_h_B * columns_h_B * sizeof(float);
    h_B = (float *) malloc(size_h_B);
    
	//@@  Initialize host memory
    const float valB = 0.01f;
    constantInit(h_A, rows_h_A * columns_h_A, 1.0f);
    constantInit(h_B, rows_h_B * columns_h_B, valB);
		
	//@@ Set numCRows and numCColumns
    rows_h_C = rows_h_A;
    columns_h_C = columns_h_B;

    //@@ Allocate the hostC matrix
	int size_h_C =  rows_h_C * columns_h_C * sizeof(float);
    h_C = (float *) malloc(size_h_C);
        
	cout << endl << "Performing CUDA computation. " ;
	    
	if ( n <= GPU_MAT_SIZE) {
		GPUsimpleMult(h_A, h_B, h_C, rows_h_A, columns_h_A, columns_h_B);
	}
	else{
		GPUroundsMult(h_A, h_B, h_C, rows_h_A, columns_h_A, columns_h_B);
	}
	
	cout << endl <<  "The dimensions of A are " <<rows_h_A << " x " << columns_h_A;
	PrintMatrix(h_A,rows_h_A,columns_h_A);
	
	cout << endl<<  "The dimensions of B are " << rows_h_B << " x " << columns_h_B;
	PrintMatrix(h_B,rows_h_B,columns_h_B);
	
	cout << endl<<  "The dimensions of C are " <<rows_h_C << " x " << columns_h_C;
	PrintMatrix(h_C,rows_h_C,columns_h_C);

	

	cout << endl<< "Checking computed result for correctness: ";
    bool correct = true;
    // test relative error by the formula 
    //     |<x, y>_cpu - <x,y>_gpu|/<|x|, |y|>  < eps 
    double eps = 1.e-6 ; // machine zero
    for (int i = 0; i < (int)(rows_h_C * columns_h_C); i++)
    {
        double abs_err = fabs(h_C[i] - (rows_h_A * valB));
        double dot_length = rows_h_A;
        double abs_val = fabs(h_C[i]);
        double rel_err = abs_err/abs_val/dot_length ;
        if (rel_err > eps)
        {
            printf("Error! Matrix[%05d]=%.8f, ref=%.8f error term is > %E\n", i, h_C[i], rows_h_A*valB, eps);
            correct = false;
        }
    }
	if(correct) cout << endl << "Result = PASS";
	else cout << endl <<"Result = FAIL";
   
    free(h_A);
    free(h_B);
    free(h_C);

	clock_end = clock();
	cout  << endl << clock_end-clock_start << " miliseconds"<< endl;

	system("pause");
    return 0;
}







Allocate, initialize host and device memory 
Performing CUDA computation. 
The dimensions of A are 10000 x 10000
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
The dimensions of B are 10000 x 10000
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
The dimensions of C are 10000 x 10000
100.002 100.002 100.002 100.002 100.002 100.002 100.002 100.002 100.002 100.002 
100.002 1

MultiBlockMultiThreadMatrixMultiplicationSharedMem.cu

In [8]:
%%cu
#define TILE_WIDTH 32 
#define GPU_MAT_SIZE 2000
#include <cuda.h>
#include <iostream>
#include <time.h>       //clock()
using namespace std;

__global__ void MatrixMulKernel( float* d_A, float* d_B, float* d_C, int m, int n, int k) { 
	__shared__ float ds_A[TILE_WIDTH][TILE_WIDTH];
	__shared__ float ds_B[TILE_WIDTH][TILE_WIDTH];
	
	int bx = blockIdx.x; int by = blockIdx.y;
	int tx = threadIdx.x; int ty = threadIdx.y;	
	int Row = by * blockDim.y + ty;
	int Col = bx * blockDim.x + tx;
	float Cvalue = 0;
	
	// Loop over the A and B tiles required to compute the Celement
	for(int t = 0; t < (n-1)/TILE_WIDTH + 1;++t) {
	
		// Collaborative loading of A tile
		if(Row < m && t*TILE_WIDTH+tx < n) {
			ds_A[ty][tx] = d_A[Row*n +t*TILE_WIDTH + tx];}
		else {
			 ds_A[ty][tx] = 0.0;
		}
		 
		// Collaborative loading of B tile
		if (t*TILE_WIDTH+ty < n && Col < k) {
			 ds_B[ty][tx] = d_B[(t*TILE_WIDTH +ty)*k + Col];
		}
		else {
			 ds_B[ty][tx] = 0.0;
		}
		
		 __syncthreads();
	    
        // Calculate the partial Cvalue in the tile
		for(int i = 0; i < TILE_WIDTH; ++i) {
			Cvalue += ds_A[ty][i] * ds_B[i][tx];
		}
		__syncthreads();
	}

	if (Row < m && Col < k)
		d_C[Row*k + Col] = Cvalue;

}

void PrintMatrix(float * M,  int numMRows, int numMColumns){
	int i, j;
	for(i = numMRows-10; i < numMRows; i ++) {
		cout << endl;
		for(j = numMColumns-10; j < numMColumns; j ++)
			cout << M[i* numMColumns+j] << " ";
	}
}

void constantInit(float *data, int size, float val)
{
    for (int i = 0; i < size; ++i)
    {
        data[i] = val;
    }
}

void subMat(float *dest, float *data,int n, int m,  int i, int j, int s)
{
    for (int k = 0; k < s; ++k)
    {
			for (int l = 0; l < s; ++l)
			{
				if ( ((k+i*s) < n)  & ((l+j*s) < m) ) { 
					dest[k*s + l] = data[((k+i*s)*m)+(l+j*s)];
				}
				else 
				{
					dest[k*s + l] = 0.0; 
				}
			}
    }
}

void putMat(float *dest, int n, int m, float *data, int i, int j, int s)
{
    for (int k = 0; k < s; ++k)
    {
			for (int l = 0; l < s; ++l)
			{
				if ( ((k+i*s) < n)  & ((l+j*s) < m) ) dest[((k+i*s)*m)+(l+j*s)] = data[k*s + l];
			}
    }
}

void addMem(float *dest, float *data1, float *data2, int size)
{
    for (int i = 0; i < size; ++i)
    {
        dest[i] = data1[i] + data2[i];
    }
}

void GPUsimpleMult(float* h_A, float* h_B, float* h_C, int m, int n, int k){
	
	float * d_A;
    float * d_B;
    float * d_C;

	int size_d_A = m * n * sizeof(float);
	int size_d_B = m * k * sizeof(float);
	int size_d_C = m * k * sizeof(float);

	
	//@@ Allocate GPU memory here
    cudaMalloc((void **) &d_A, size_d_A);
	cudaMalloc((void **) &d_B, size_d_B);
	cudaMalloc((void **) &d_C, size_d_C);
	
    //@@ Copy memory to the GPU here
    cudaMemcpy(d_A, h_A, size_d_A, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size_d_B, cudaMemcpyHostToDevice);
	
	//@@ Initialize the grid and block dimensions here
    dim3 dimGrid((m-1)/TILE_WIDTH+1, (k-1)/TILE_WIDTH+1, 1);
    dim3 dimBlock(TILE_WIDTH, TILE_WIDTH, 1);
	
    //@@ Launch the GPU Kernel here
    MatrixMulKernel<<<dimGrid, dimBlock>>>(d_A,d_B,d_C,m,n,k);  
    cudaThreadSynchronize();
 	
	
	
    //@@ Copy the GPU memory back to the CPU here
    cudaMemcpy(h_C, d_C, size_d_C, cudaMemcpyDeviceToHost);
	
    //@@ Free the GPU memory here
    cudaFree(d_A); 
    cudaFree(d_B); 
    cudaFree(d_C);
  	
}


void GPUroundsMult(float* h_A, float* h_B, float* h_C, int m, int n, int k){
	
	float * d_A;
    float * d_B;
    float * d_C;

	float * h_A_aux; 
	float * h_B_aux; 
	float * h_C_aux0; 
	float * h_C_aux1; 

	int size_d = GPU_MAT_SIZE * GPU_MAT_SIZE * sizeof(float);
	
	//@@ Allocate GPU memory here
	h_A_aux = (float *) malloc( size_d);
	h_B_aux = (float *) malloc( size_d);
	h_C_aux0 = (float *) malloc( size_d);
	h_C_aux1 = (float *) malloc( size_d);
    
	cudaMalloc((void **) &d_A, size_d);
	cudaMalloc((void **) &d_B, size_d);
	cudaMalloc((void **) &d_C, size_d);
	
	for (int i = 0; i < (m-1)/GPU_MAT_SIZE+1; ++i){ 
		
		for (int j = 0; j < (k-1)/GPU_MAT_SIZE+1; ++j){
			
			constantInit(h_C_aux0, GPU_MAT_SIZE * GPU_MAT_SIZE, 0.0);

			for (int l = 0; l < (n-1)/GPU_MAT_SIZE+1; ++l){

					subMat(h_A_aux, h_A,n,m, i, l, GPU_MAT_SIZE);
					subMat(h_B_aux, h_B,m,k, l, j, GPU_MAT_SIZE);
					GPUsimpleMult(h_A_aux, h_B_aux, h_C_aux1, GPU_MAT_SIZE,GPU_MAT_SIZE,GPU_MAT_SIZE);
					addMem(h_C_aux0,h_C_aux0,h_C_aux1,GPU_MAT_SIZE * GPU_MAT_SIZE);
			}

			putMat(h_C, m, k, h_C_aux0, i, j,GPU_MAT_SIZE );
		}       
	}

	//@@ Free the aux CPU and GPU memory here
    cudaFree(d_A); 
    cudaFree(d_B); 
    cudaFree(d_C);
  	free(h_A_aux);
    free(h_B_aux);
    free(h_C_aux0);
	free(h_C_aux1);

}
int main(int argc, char ** argv) {
	clock_t clock_start,clock_end;
    clock_start = clock();

	float * h_A; // The A matrix
    float * h_B; // The B matrix
    float * h_C; // The output C matrix
    
    int rows_h_A; // number of rows in the matrix A
    int columns_h_A; // number of columns in the matrix A
    int rows_h_B; // number of rows in the matrix B
    int columns_h_B; // number of columns in the matrix B
    int rows_h_C; // number of rows in the matrix C (you have to set this)
    int columns_h_C; // number of columns in the matrix C (you have to set this)
	
	int n = 10000;
	rows_h_A = n; // number of rows in the matrix A
    columns_h_A = n; // number of columns in the matrix A
    rows_h_B = n; // number of rows in the matrix B
	columns_h_B = n; // number of columns in the matrix B
	
     cout << endl << "Allocate, initialize host and device memory " ;
	//@@ Allocate host memory
	int size_h_A = rows_h_A * columns_h_A * sizeof(float);
	h_A = (float *) malloc(size_h_A);
	int size_h_B = rows_h_B * columns_h_B * sizeof(float);
    h_B = (float *) malloc(size_h_B);
    
	//@@  Initialize host memory
    const float valB = 0.01f;
    constantInit(h_A, rows_h_A * columns_h_A, 1.0f);
    constantInit(h_B, rows_h_B * columns_h_B, valB);
		
	//@@ Set numCRows and numCColumns
    rows_h_C = rows_h_A;
    columns_h_C = columns_h_B;

    //@@ Allocate the hostC matrix
	int size_h_C =  rows_h_C * columns_h_C * sizeof(float);
    h_C = (float *) malloc(size_h_C);
        
	cout << endl << "Performing CUDA computation. " ;
	    
	if ( n <= GPU_MAT_SIZE) {
		GPUsimpleMult(h_A, h_B, h_C, rows_h_A, columns_h_A, columns_h_B);
	}
	else{
		GPUroundsMult(h_A, h_B, h_C, rows_h_A, columns_h_A, columns_h_B);
	}
	
	cout << endl <<  "The dimensions of A are " <<rows_h_A << " x " << columns_h_A;
	PrintMatrix(h_A,rows_h_A,columns_h_A);
	
	cout << endl<<  "The dimensions of B are " << rows_h_B << " x " << columns_h_B;
	PrintMatrix(h_B,rows_h_B,columns_h_B);
	
	cout << endl<<  "The dimensions of C are " <<rows_h_C << " x " << columns_h_C;
	PrintMatrix(h_C,rows_h_C,columns_h_C);

	

	cout << endl<< "Checking computed result for correctness: ";
    bool correct = true;
    // test relative error by the formula 
    //     |<x, y>_cpu - <x,y>_gpu|/<|x|, |y|>  < eps 
    double eps = 1.e-6 ; // machine zero
    for (int i = 0; i < (int)(rows_h_C * columns_h_C); i++)
    {
        double abs_err = fabs(h_C[i] - (rows_h_A * valB));
        double dot_length = rows_h_A;
        double abs_val = fabs(h_C[i]);
        double rel_err = abs_err/abs_val/dot_length ;
        if (rel_err > eps)
        {
            printf("Error! Matrix[%05d]=%.8f, ref=%.8f error term is > %E\n", i, h_C[i], rows_h_A*valB, eps);
            correct = false;
        }
    }
	if(correct) cout << endl << "Result = PASS";
	else cout << endl <<"Result = FAIL";
   
    free(h_A);
    free(h_B);
    free(h_C);

	clock_end = clock();
	cout  << endl << clock_end-clock_start << " miliseconds"<< endl;

	system("pause");
    return 0;
}


Allocate, initialize host and device memory 
Performing CUDA computation. 
The dimensions of A are 10000 x 10000
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
The dimensions of B are 10000 x 10000
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
The dimensions of C are 10000 x 10000
100.002 100.002 100.002 100.002 100.002 100.002 100.002 100.002 100.002 100.002 
100.002 1

MultiBlockMultiThreadMatrixMultiplicationTransp.cu

In [9]:
%%cu
#define TILE_WIDTH 32 
#define GPU_MAT_SIZE 2000
#include <cuda.h>
#include <iostream>
#include <time.h>       //clock()
using namespace std;

__global__ void MatrixMulKernel( float* d_A, float* d_B, float* d_C, int m, int n, int k) { 
	
	int Row = blockIdx.y*blockDim.y + threadIdx.y;
    int Col = blockIdx.x*blockDim.x + threadIdx.x;
	
    if ((Row < m) && (Col < k)) {
       float Cvalue = 0.0;
       for (int i = 0; i < n; ++i)
            Cvalue += d_A[Row*n+i] * d_B[Col*n+i];
       d_C[Row*k+Col] = Cvalue;
     }
	     
}

void TranspMatrix( float* M, int numMRows, int numMColumns, float* MTr, int numMTrRows, int numMTrColumns) { 
	if (( numMRows != numMTrColumns) && ( numMColumns != numMTrRows)){
		cout << "non compatible transposition " << endl;
		return;
	}

	int i, j;
	for(i = 0; i < numMRows; i ++){
		for(j = 0; j < numMColumns; j ++){
           MTr[j * numMTrColumns + i] = M[i * numMColumns + j];
       }
	}
}

void PrintMatrix(float * M,  int numMRows, int numMColumns){
	int i, j;
	for(i = numMRows-10; i < numMRows; i ++) {
		cout << endl;
		for(j = numMColumns-10; j < numMColumns; j ++)
			cout << M[i* numMColumns+j] << " ";
	}
}

void constantInit(float *data, int size, float val)
{
    for (int i = 0; i < size; ++i)
    {
        data[i] = val;
    }
}

void subMat(float *dest, float *data,int n, int m,  int i, int j, int s)
{
    for (int k = 0; k < s; ++k)
    {
			for (int l = 0; l < s; ++l)
			{
				if ( ((k+i*s) < n)  & ((l+j*s) < m) ) { 
					dest[k*s + l] = data[((k+i*s)*m)+(l+j*s)];
				}
				else 
				{
					dest[k*s + l] = 0.0; 
				}
			}
    }
}

void putMat(float *dest, int n, int m, float *data, int i, int j, int s)
{
    for (int k = 0; k < s; ++k)
    {
			for (int l = 0; l < s; ++l)
			{
				if ( ((k+i*s) < n)  & ((l+j*s) < m) ) dest[((k+i*s)*m)+(l+j*s)] = data[k*s + l];
			}
    }
}

void addMem(float *dest, float *data1, float *data2, int size)
{
    for (int i = 0; i < size; ++i)
    {
        dest[i] = data1[i] + data2[i];
    }
}

void GPUsimpleMult(float* h_A, float* h_B, float* h_C, int m, int n, int k){
	
	float * d_A;
    float * d_B;
    float * d_C;

	int size_d_A = m * n * sizeof(float);
	int size_d_B = m * k * sizeof(float);
	int size_d_C = m * k * sizeof(float);

	
	//@@ Allocate GPU memory here
    cudaMalloc((void **) &d_A, size_d_A);
	cudaMalloc((void **) &d_B, size_d_B);
	cudaMalloc((void **) &d_C, size_d_C);
	
    //@@ Copy memory to the GPU here
    cudaMemcpy(d_A, h_A, size_d_A, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size_d_B, cudaMemcpyHostToDevice);
	
	//@@ Initialize the grid and block dimensions here
    dim3 dimGrid((m-1)/TILE_WIDTH+1, (k-1)/TILE_WIDTH+1, 1);
    dim3 dimBlock(TILE_WIDTH, TILE_WIDTH, 1);
	
    //@@ Launch the GPU Kernel here
    MatrixMulKernel<<<dimGrid, dimBlock>>>(d_A,d_B,d_C,m,n,k);  
    cudaThreadSynchronize();
 	
	
	
    //@@ Copy the GPU memory back to the CPU here
    cudaMemcpy(h_C, d_C, size_d_C, cudaMemcpyDeviceToHost);
	
    //@@ Free the GPU memory here
    cudaFree(d_A); 
    cudaFree(d_B); 
    cudaFree(d_C);
  	
}


void GPUroundsMult(float* h_A, float* h_B, float* h_C, int m, int n, int k){
	
	float * d_A;
    float * d_B;
    float * d_C;

	float * h_A_aux; 
	float * h_B_aux; 
	float * h_C_aux0; 
	float * h_C_aux1; 

	int size_d = GPU_MAT_SIZE * GPU_MAT_SIZE * sizeof(float);
	
	//@@ Allocate GPU memory here
	h_A_aux = (float *) malloc( size_d);
	h_B_aux = (float *) malloc( size_d);
	h_C_aux0 = (float *) malloc( size_d);
	h_C_aux1 = (float *) malloc( size_d);
    
	cudaMalloc((void **) &d_A, size_d);
	cudaMalloc((void **) &d_B, size_d);
	cudaMalloc((void **) &d_C, size_d);
	
	for (int i = 0; i < (m-1)/GPU_MAT_SIZE+1; ++i){ 
		
		for (int j = 0; j < (k-1)/GPU_MAT_SIZE+1; ++j){
			
			constantInit(h_C_aux0, GPU_MAT_SIZE * GPU_MAT_SIZE, 0.0);

			for (int l = 0; l < (n-1)/GPU_MAT_SIZE+1; ++l){

					subMat(h_A_aux, h_A,n,m, i, l, GPU_MAT_SIZE);
					subMat(h_B_aux, h_B,m,k, l, j, GPU_MAT_SIZE);
					GPUsimpleMult(h_A_aux, h_B_aux, h_C_aux1, GPU_MAT_SIZE,GPU_MAT_SIZE,GPU_MAT_SIZE);
					addMem(h_C_aux0,h_C_aux0,h_C_aux1,GPU_MAT_SIZE * GPU_MAT_SIZE);
			}

			putMat(h_C, m, k, h_C_aux0, i, j,GPU_MAT_SIZE );
		}       
	}

	//@@ Free the aux CPU and GPU memory here
    cudaFree(d_A); 
    cudaFree(d_B); 
    cudaFree(d_C);
  	free(h_A_aux);
    free(h_B_aux);
    free(h_C_aux0);
	free(h_C_aux1);

}
int main(int argc, char ** argv) {
	clock_t clock_start,clock_end;
    clock_start = clock();

	float * h_A; // The A matrix
    float * h_B; // The B matrix
	float * h_BT; // The B matrix
    float * h_C; // The output C matrix
    
    int rows_h_A; // number of rows in the matrix A
    int columns_h_A; // number of columns in the matrix A
    int rows_h_B; // number of rows in the matrix B
    int columns_h_B; // number of columns in the matrix B
    int rows_h_C; // number of rows in the matrix C (you have to set this)
    int columns_h_C; // number of columns in the matrix C (you have to set this)
	int rows_h_BT; // number of rows in the matrix B transposed
    int columns_h_BT; // number of columns in the matrix B transposed
	
	int n = 1000;
	rows_h_A = n; // number of rows in the matrix A
    columns_h_A = n; // number of columns in the matrix A
    rows_h_B = n; // number of rows in the matrix B
	columns_h_B = n; // number of columns in the matrix B
	rows_h_BT = n; // number of rows in the matrix B
	columns_h_BT = n; // number of columns in the matrix B
	
     cout << endl << "Allocate, initialize host and device memory " ;
	//@@ Allocate host memory
	int size_h_A = rows_h_A * columns_h_A * sizeof(float);
	h_A = (float *) malloc(size_h_A);
	int size_h_B = rows_h_B * columns_h_B * sizeof(float);
    h_B = (float *) malloc(size_h_B);
    int size_h_BT = rows_h_BT * columns_h_BT * sizeof(float);
    h_BT = (float *) malloc(size_h_BT);
	
	//@@  Initialize host memory
    const float valB = 0.01f;
    constantInit(h_A, rows_h_A * columns_h_A, 1.0f);
    constantInit(h_B, rows_h_B * columns_h_B, valB);
	TranspMatrix(h_B,rows_h_B, columns_h_B,h_BT,rows_h_BT, columns_h_BT);
		
	//@@ Set numCRows and numCColumns
    rows_h_C = rows_h_A;
    columns_h_C = columns_h_B;

    //@@ Allocate the hostC matrix
	int size_h_C =  rows_h_C * columns_h_C * sizeof(float);
    h_C = (float *) malloc(size_h_C);
        
	cout << endl << "Performing CUDA computation. " ;
	    
	if ( n <= GPU_MAT_SIZE) {
		GPUsimpleMult(h_A, h_BT, h_C, rows_h_A, columns_h_A, columns_h_BT);
	}
	else{
		GPUroundsMult(h_A, h_BT, h_C, rows_h_A, columns_h_A, columns_h_BT);
	}
	
	cout << endl <<  "The dimensions of A are " <<rows_h_A << " x " << columns_h_A;
	PrintMatrix(h_A,rows_h_A,columns_h_A);
	
	cout << endl<<  "The dimensions of B are " << rows_h_B << " x " << columns_h_B;
	PrintMatrix(h_B,rows_h_B,columns_h_B);
	
	cout << endl<<  "The dimensions of C are " <<rows_h_C << " x " << columns_h_C;
	PrintMatrix(h_C,rows_h_C,columns_h_C);

	

	cout << endl<< "Checking computed result for correctness: ";
    bool correct = true;
    // test relative error by the formula 
    //     |<x, y>_cpu - <x,y>_gpu|/<|x|, |y|>  < eps 
    double eps = 1.e-6 ; // machine zero
    for (int i = 0; i < (int)(rows_h_C * columns_h_C); i++)
    {
        double abs_err = fabs(h_C[i] - (rows_h_A * valB));
        double dot_length = rows_h_A;
        double abs_val = fabs(h_C[i]);
        double rel_err = abs_err/abs_val/dot_length ;
        if (rel_err > eps)
        {
            printf("Error! Matrix[%05d]=%.8f, ref=%.8f error term is > %E\n", i, h_C[i], rows_h_A*valB, eps);
            correct = false;
        }
    }
	if(correct) cout << endl << "Result = PASS";
	else cout << endl <<"Result = FAIL";
   
    free(h_A);
    free(h_B);
    free(h_C);

	clock_end = clock();
	cout  << endl << clock_end-clock_start << " miliseconds"<< endl;

	system("pause");
    return 0;
}



Allocate, initialize host and device memory 
Performing CUDA computation. 
The dimensions of A are 1000 x 1000
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 1 1 1 
The dimensions of B are 1000 x 1000
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 
The dimensions of C are 1000 x 1000
10.0001 10.0001 10.0001 10.0001 10.0001 10.0001 10.0001 10.0001 10.0001 10.0001 
10.0001 10.0001